## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 3349 (12bits), N的範圍: 0 ~ 4095 (12bits)

### Define function
* (Not including 2's complement range)

In [1]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


### 查找表範例

In [2]:
def create_mapping(source_list, address_map):
    """
    建立值到位址和位址到值的雙向映射。
    若 address_map 中某位址沒有對應值，則該位址的值為 0。
    
    參數:
    source_list (list): 原始資料列表。
    address_map (list): 位址映射列表。
    
    回傳:
    tuple: (值到位址的映射, 位址到值的映射)
    """
    # 初始化值到位址的映射
    value_to_address = {value: addr for value, addr in zip(source_list, address_map)}
    
    # 初始化位址到值的映射，未指定的位址預設為 0
    max_address = max(address_map)  # 確保涵蓋所有位址
    address_to_value = {addr: 0 for addr in range(max_address + 1)}  # 預設為 0
    for value, addr in zip(source_list, address_map):
        address_to_value[addr] = value  # 更新有指定值的位址

    return value_to_address, address_to_value

# 測試數據
source_list = [10, 20, 30]
address_map = [2, 0, 4]

# 建立映射
value_to_address, address_to_value = create_mapping(source_list, address_map)

# 顯示結果
print("值到位址的映射:", value_to_address)
print("位址到值的映射:", address_to_value)


# 測試數據
source_list = [10, 20, 30, 40]
address_map = [2, 0, 3, 1]

# 建立映射
value_to_address, address_to_value = create_mapping(source_list, address_map)

# 測試查詢
print("值到位址:", value_to_address)  # 查詢 10 -> 2, 20 -> 0
print("位址到值:", address_to_value)  # 查詢 2 -> 10, 0 -> 20

# 範例查詢
value = 10
address = 2
print(f"輸入值 {value}, 對應位址: {value_to_address[value]}")
print(f"輸入位址 {address}, 對應值: {address_to_value[address]}")

值到位址的映射: {10: 2, 20: 0, 30: 4}
位址到值的映射: {0: 20, 1: 0, 2: 10, 3: 0, 4: 30}
值到位址: {10: 2, 20: 0, 30: 3, 40: 1}
位址到值: {0: 20, 1: 40, 2: 10, 3: 30}
輸入值 10, 對應位址: 2
輸入位址 2, 對應值: 10


## 以12bits為例:

In [3]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data = 4095     ## 
AN = 3349
input = data*AN
bits = bits_needed(input)
print("AN資料的位元數:", bits)

AN資料的位元數: 24


### 產生1-bit AWE (Arithmetic-Weight Error)

In [4]:
AWE_1=[]
e1=0     ## 錯誤數量
e1_data=[]
for i in range(0, bits):
    AWE_1.insert(e1 , input + 2**i)
    e1_data.insert(e1, 2**i)
    e1 = e1 + 1
    AWE_1.insert(e1 , input - 2**i)
    e1_data.insert(e1, -2**i)
    e1 = e1 + 1

print(bits)
print(input)
print("產生1-bit AWE:",AWE_1)
print("算術錯誤的內容:", e1_data)
print(e1)
print(np.size(AWE_1))

24
13714155
產生1-bit AWE: [13714156, 13714154, 13714157, 13714153, 13714159, 13714151, 13714163, 13714147, 13714171, 13714139, 13714187, 13714123, 13714219, 13714091, 13714283, 13714027, 13714411, 13713899, 13714667, 13713643, 13715179, 13713131, 13716203, 13712107, 13718251, 13710059, 13722347, 13705963, 13730539, 13697771, 13746923, 13681387, 13779691, 13648619, 13845227, 13583083, 13976299, 13452011, 14238443, 13189867, 14762731, 12665579, 15811307, 11617003, 17908459, 9519851, 22102763, 5325547]
算術錯誤的內容: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608]
48
48


### 產生SEC的模數:

In [61]:
def custom_mod(a, b):
    if b == 0:
        return "錯誤：除數不能為零！"
    # 計算商和餘數，修正餘數計算邏輯
    quotient = a // b
    remainder = a - b * quotient
    # 修正負餘數的情況
    if remainder > 0 and a < 0:
        remainder -= abs(b)
        quotient += 1
    return remainder

SEC_R = []
for m in range(0,e1):
    mod = AWE_1[m] % AN
    SEC_R.append(mod)

print("SEC的模數:\n",SEC_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 3348, 2, 3347, 4, 3345, 8, 3341, 16, 3333, 32, 3317, 64, 3285, 128, 3221, 256, 3093, 512, 2837, 1024, 2325, 2048, 1301, 747, 2602, 1494, 1855, 2988, 361, 2627, 722, 1905, 1444, 461, 2888, 922, 2427, 1844, 1505, 339, 3010, 678, 2671, 1356, 1993, 2712, 637]
SEC的大小: 48


## 建立只能做SEC的LUT

In [79]:
AWE_to_SEC_R, SEC_R_to_AWE = create_mapping(e1_data, SEC_R)

### 產生2-bit AWE (先固定某位元)

In [73]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
N = 15
SEC_value = +2**N     ## 先取某一固定位元,AWE為: (+) or (-) 2^N
print("對應的餘數:",AWE_to_SEC_R[SEC_value])
for j in range(0, bits):
    e2_data.insert(e2 , (input + SEC_value + 2**j))
    AWE_2.insert(e2 , (SEC_value + 2**j))        
    print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
    print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
    e2 = e2 + 1
    e2_data.insert(e2 , (input + SEC_value - 2**j))
    AWE_2.insert(e2 , (SEC_value - 2**j))        
    print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
    print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
    e2 = e2 + 1
    p = p + 1
####################################################
del AWE_2[(2*N)+1]
del AWE_2[2*N]
del e2_data[(2*N)+1]
del e2_data[2*N]
####################################################
p = p -1
e2 = e2 - 2
print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits e2_data:",e2_data)

對應的餘數: 2627
第 0 筆錯誤資料: 13746924
第 0 筆算術錯誤: 32769
第 1 筆錯誤資料: 13746922
第 1 筆算術錯誤: 32767
第 2 筆錯誤資料: 13746925
第 2 筆算術錯誤: 32770
第 3 筆錯誤資料: 13746921
第 3 筆算術錯誤: 32766
第 4 筆錯誤資料: 13746927
第 4 筆算術錯誤: 32772
第 5 筆錯誤資料: 13746919
第 5 筆算術錯誤: 32764
第 6 筆錯誤資料: 13746931
第 6 筆算術錯誤: 32776
第 7 筆錯誤資料: 13746915
第 7 筆算術錯誤: 32760
第 8 筆錯誤資料: 13746939
第 8 筆算術錯誤: 32784
第 9 筆錯誤資料: 13746907
第 9 筆算術錯誤: 32752
第 10 筆錯誤資料: 13746955
第 10 筆算術錯誤: 32800
第 11 筆錯誤資料: 13746891
第 11 筆算術錯誤: 32736
第 12 筆錯誤資料: 13746987
第 12 筆算術錯誤: 32832
第 13 筆錯誤資料: 13746859
第 13 筆算術錯誤: 32704
第 14 筆錯誤資料: 13747051
第 14 筆算術錯誤: 32896
第 15 筆錯誤資料: 13746795
第 15 筆算術錯誤: 32640
第 16 筆錯誤資料: 13747179
第 16 筆算術錯誤: 33024
第 17 筆錯誤資料: 13746667
第 17 筆算術錯誤: 32512
第 18 筆錯誤資料: 13747435
第 18 筆算術錯誤: 33280
第 19 筆錯誤資料: 13746411
第 19 筆算術錯誤: 32256
第 20 筆錯誤資料: 13747947
第 20 筆算術錯誤: 33792
第 21 筆錯誤資料: 13745899
第 21 筆算術錯誤: 31744
第 22 筆錯誤資料: 13748971
第 22 筆算術錯誤: 34816
第 23 筆錯誤資料: 13744875
第 23 筆算術錯誤: 30720
第 24 筆錯誤資料: 13751019
第 24 筆算術錯誤: 36864
第 25 筆錯誤資料: 13742827
第 25 筆算術錯誤: 

### 產生DEC模數: (先固定某一位元為例)

In [80]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))


DEC_R的模數:
 [2628, 2626, 2629, 2625, 2631, 2623, 2635, 2619, 2643, 2611, 2659, 2595, 2691, 2563, 2755, 2499, 2883, 2371, 3139, 2115, 302, 1603, 1326, 579, 25, 1880, 772, 1133, 2266, 2988, 1183, 722, 3088, 2166, 200, 1705, 1122, 783, 2966, 2288, 3305, 1949, 634, 1271, 1990, 3264]
46


### 尋找SEC和DEC關係:

In [75]:
SEC_R2 = []
position = 0
for i in range(len(DEC_R)):
    SEC_R2.insert(position, (DEC_R[i] - AWE_to_SEC_R[SEC_value]))  ##  R2 = R - R1
    position = position+1
    
for i in range(len(SEC_R2)):
    if SEC_R2[i] < 0:
        SEC_R2[i] = AN + SEC_R2[i]    ## 若值小於0, AN+負值
   
print(SEC_R2) 
print(len(SEC_R2))


[1, 3348, 2, 3347, 4, 3345, 8, 3341, 16, 3333, 32, 3317, 64, 3285, 128, 3221, 256, 3093, 512, 2837, 1024, 2325, 2048, 1301, 747, 2602, 1494, 1855, 2988, 361, 1905, 1444, 461, 2888, 922, 2427, 1844, 1505, 339, 3010, 678, 2671, 1356, 1993, 2712, 637]
46


In [76]:
SEC_AWE2 = [] 
position = 0
for A in range(len(SEC_R2)):
    SEC_AWE2.insert(position,(SEC_R_to_AWE[SEC_R2[A]]))
    position = position + 1

print(SEC_AWE2)

[1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608]


### Trade-off algorithm:(以下為大致流程)

![Trade-off演算法](./圖片/Trade-off%20algorithm.png )

In [77]:
## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list

In [81]:
###給定一個DEC_R, 找SEC_R1及SEC_R2
import math
import time

# print("產生2-bits AWE:\n",AWE_2)
# print("Double AWE的餘數:\n",DEC_R)
AWE1 = 0 ; AWE2 = 0; R1 = 0; R2 = 0
possible_AWE = []; possible_AWE1 = []; possible_AWE2 = []

start_time = time.time()  # 記錄開始時間
for R in DEC_R:
    for errors in range(0,bits):
        for s in range(-1,2,2):                         ### 0:+, 1:-
            AWE1 = s*(2**errors)
            R1 = AWE_to_SEC_R[AWE1]
            R2 = R - R1
            if R2 < 0: 
                R2 = AN + R2
            AWE2 = SEC_R_to_AWE[R2]               ### 若找不到對應的AWE2,
            if AWE2 != 0:                         ### AWE2定為0(函數內容已定義)  
                possible_AWE.append(AWE1+AWE2)
                #possible_AWE1.append(AWE1)
                #possible_AWE2.append(AWE1)
            
possible_AWE = remove_duplicates(possible_AWE)    ### 把重複的AWE去除                
end_time = time.time()  # 記錄結束時間
elapsed_time = end_time - start_time  # 計算執行時間
            
print(f"執行時間: {elapsed_time:.6f} 秒")     
print("用折衷法 (Trade-off) 解出來的 Double AWE:\n",possible_AWE)
print("是否和產生2-bits AWE相同:",possible_AWE == AWE_2)
print(elapsed_time / np.size(possible_AWE) )

執行時間: 0.001001 秒
用折衷法 (Trade-off) 解出來的 Double AWE:
 [32769, 32767, 32770, 32766, 32772, 32764, 32776, 32760, 32784, 32752, 32800, 32736, 32832, 32704, 32896, 32640, 33024, 32512, 33280, 32256, 33792, 31744, 34816, 30720, 36864, 28672, 40960, 24576, 49152, 16384, 98304, -32768, 163840, -98304, 294912, -229376, 557056, -491520, 1081344, -1015808, 2129920, -2064384, 4227072, -4161536, 8421376, -8355840]
是否和產生2-bits AWE相同: True
2.1763469861901324e-05
